In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np
import os


In [11]:

# Define the CNN model
class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(2, 2),

            nn.Conv2d(32, 32, 3, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(2, 2),

            nn.Conv2d(32, 32, 3, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(2, 2),

            nn.Flatten(),
            nn.Linear(32, 10)
        )


    def forward(self, x):
        x = self.network(x)
        return x



In [12]:
def val(net, device, data_loader, T=None):
    net.eval().to(device)
    correct = 0.0
    total = 0.0
    if T is not None:
        corrects = np.zeros(T)
    with torch.no_grad():
        for batch, (img, label) in enumerate(tqdm(data_loader)):
            img = img.to(device)
            if T is None:
                out = net(img)
                correct += (out.argmax(dim=1) == label.to(device)).float().sum().item()
            else:
                for m in net.modules():
                    if hasattr(m, 'reset'):
                        m.reset()
                for t in range(T):
                    if t == 0:
                        out = net(img)
                    else:
                        out += net(img)
                    corrects[t] += (out.argmax(dim=1) == label.to(device)).float().sum().item()
            total += out.shape[0]
    return correct / total if T is None else corrects / total

In [13]:
torch.random.manual_seed(0)
torch.cuda.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_dir = '/content/drive/MyDrive/Dissertation/project_code/Dataset/mnist'
path = '/content/drive/MyDrive/Dissertation/project_code/ann_models/'
batch_size = 100
T = 50
lr = 1e-3
epochs = 10

model = MNIST_CNN().to(device)
train_data_dataset = torchvision.datasets.MNIST(
    root=dataset_dir,
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True)
train_data_loader = torch.utils.data.DataLoader(
    dataset=train_data_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False)
test_data_dataset = torchvision.datasets.MNIST(
    root=dataset_dir,
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True)
test_data_loader = torch.utils.data.DataLoader(
    dataset=test_data_dataset,
    batch_size=50,
    shuffle=True,
    drop_last=False)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)

for epoch in range(epochs):
    model.train()
    for (img, label) in train_data_loader:
        optimizer.zero_grad()
        out = model(img.to(device))
        loss = loss_function(out, label.to(device))
        loss.backward()
        optimizer.step()
    torch.save(model.state_dict(), os.path.join(path + 'cnn_model.pth'))
    print('Epoch: %d' % epoch)
    acc = val(model, device, train_data_loader)
    print('Validating Accuracy: %.3f' % (acc))
    print()

acc = val(model, device, test_data_loader)

print('ANN Validating Accuracy: %.4f' % (acc))


print('Finished Training')

# Save the trained model to disk
torch.save(model, os.path.join(path + 'cnn_model.h5'))
torch.save(model.state_dict(), os.path.join(path + 'cnn_model.pth'))
torch.save(model.state_dict(), os.path.join(path + 'cnn_model.pkl'))

Epoch: 0


100%|██████████| 600/600 [00:28<00:00, 21.43it/s]


Validating Accuracy: 0.977

Epoch: 1


100%|██████████| 600/600 [00:27<00:00, 21.80it/s]


Validating Accuracy: 0.985

Epoch: 2


...

Validating Accuracy: 0.991

Epoch: 9


100%|██████████| 600/600 [00:25<00:00, 23.30it/s]


Validating Accuracy: 0.993



100%|██████████| 200/200 [00:06<00:00, 32.45it/s]


ANN Validating Accuracy: 0.9882
Finished Training
